# 问题描述
用复化梯形公式$T_{n}=h\left(\frac{1}{2} f(a)+\sum_{j=1}^{n} f(a+i h)+\frac{1}{2} f(b)\right)$的自动控制误差算法求积分$I=\int_{a}^{b} f(x) d x$
* 说明：
下面的程序编写中以$\frac{\sin (x)}{x}$作为被积函数，计算积分
$$\int_{0}^{1} \frac{\sin (x)}{x}$$
要求计算精度满足$\left|T_{2 n}-T_{n}\right|<\varepsilon=10^{-7}$

# 求解

In [1]:
import math
def f(x):
    """f(x)表示被积函数"""
    return math.sin(x)/x

In [2]:
#图像绘制
import numpy as np, matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
x_ = np.arange(0,1,0.01)
y_=[]
for i in x_:
    y_.append(f(i))
y_[0]=1


fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(x_, y_)
ax.fill_between(x_,y_, color='green', alpha=0.5)
ax.set_xlabel("$x$", fontsize=18)
ax.set_ylabel("$f(x)$", fontsize=18)
ax.set_ylim(0, 1.2)
plt.show()

C:\Users\13738\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


<Figure size 800x600 with 1 Axes>

## 梯形法递推求解

In [3]:
def variableStepLadder(a,b,err):
    """a,b为积分边界,
        err为精度"""
    h=b-a
    n=1    #初始化子区间的个数
    t2n=h*(1+f(1))/2
    while True:
        tn=t2n
        h=h/2
        t2n=0
        for i in range(0,n):
            x=a+h*(2*i+1)
            t2n=t2n+f(x)
        t2n=tn/2+h*t2n
        n=n*2
        #满足精度时就退出积分
        if abs(t2n-tn)<err:
            break
    print(n)
    return t2n

In [4]:
%%time
err=0.0000001
ans=variableStepLadder(0,1,err)
print("控制精度为"+str(err)+"时，梯形递推积分结果为："+str(ans))

1024
控制精度为1e-07时，梯形递推积分结果为：0.946083046432447
Wall time: 0 ns


## Romberg积分算法求积分

In [5]:
import numpy as np
def romberg(a,b,err):
    """龙贝格算法求解
        a,b为积分边界,
        err为精度"""
    nsize=24    #外推次数
    r=np.zeros((nsize,nsize))    #r为T表，初始为0
    h=(b-a)/2
    n=1    #子区间的个数
    r[(0,0)]=h*(1+f(b))    #f(a)的值为1，f(b)的值调用函数求解
    for i in range(nsize):
        #逐次分半梯形公式求积
        s=0
        for j in range(n):
            x=a+h*(2*j+1)
            s=s+f(x)
        r[(i,0)]=r[(i-1,0)]/2+h*s
        n=n*2
        h=h/2
        #Romberg加速
        m=1
        for j in range(1,i+1):
            m=m*4
            r[(i-j,j)]=r[(i-j+1,j-1)]+(r[(i-j+1,j-1)]-r[(i-j,j-1)])/(m-1)
        if abs(r[(0,i)]-r[(0,i-1)])<=err:
            break
    abs(r[(0,i)]-r[(0,i-1)])
    if i<nsize-1:
        return r[(0,i)]
    else:
        return -1    #迭代次数达到上限

In [6]:
%%time
err=0.0000001
ans=romberg(0,1,err)
if ans>=0:
    print("龙贝格求积结果:"+str(ans))
else:
    print("迭代次数达到上限，函数退出")

龙贝格求积结果:0.9460830035068026
Wall time: 3.28 s


# 标准结果
通过scipy库中的`integrate.quad(f, a, b)`函数返回的值作为标准结果

In [7]:
from scipy import integrate
v, err = integrate.quad(f, 0, 1)
print("参考结果为:"+str(v))


参考结果为:0.9460830703671831
